In [ ]:
import os
import pandas as pd
import gradio as gr
import unicodedata

# Parámetros de configuración
maximo_lineas = 35      # Ancho mínimo para cada columna (en píxeles)
altura = maximo_lineas * 35  # Altura en píxeles para los componentes de código
lenguaje_programacion = ".java" # lenguaje de programación que vamos a corregir.
fichero_correciones_csv = "Calificaciones.csv" # Fichero que descargamos de Moodle de la actividad

### FUNCIONES AUXILIARES Y DE GESTIÓN (se mantienen igual) ###
def format_grade(grade: str) -> str:
    try:
        num = float(grade.replace(',', '.'))
        return f"{num:.2f}".replace('.', ',')
    except Exception as e:
        return grade

def normalize_string(s: str) -> str:
    s = s.lower()
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def obtener_nota(carpeta: str, archivo_seleccionado: str) -> str:
    csv_path = os.path.join(carpeta, fichero_correciones_csv)
    if not os.path.exists(csv_path):
        return ""
    try:
        df = pd.read_csv(csv_path, encoding='utf-8-sig')
    except Exception as e:
        return ""
    student_name = archivo_seleccionado.split("_")[0].strip()
    norm_student = normalize_string(student_name)
    df["norm_nombre"] = df["Nombre completo"].apply(lambda x: normalize_string(str(x)))
    matching = df[df["norm_nombre"] == norm_student]
    if not matching.empty:
        nota = str(matching.iloc[0]["Calificación"])
        return nota
    return ""

def listar_archivos_prueba(carpeta: str):
    if not os.path.isdir(carpeta):
         return gr.update(choices=[], value=None), ""
    # Se excluye "Solucion.java" de la lista
    java_files = sorted([f for f in os.listdir(carpeta) if f.endswith(lenguaje_programacion) and f != f"Solucion{lenguaje_programacion}"])
    if not java_files:
         return gr.update(choices=[], value=None), ""
    csv_path = os.path.join(carpeta, fichero_correciones_csv)
    if os.path.exists(csv_path):
         try:
             _ = cargar_csv(carpeta)
         except Exception as e:
             pass
         nota = obtener_nota(carpeta, java_files[0])
    else:
         nota = ""
    return gr.update(choices=java_files, value=java_files[0]), nota

def cargar_archivos_desde_carpeta(carpeta: str, archivo_seleccionado: str):
    # Cargar el archivo Java (examen del estudiante)
    java_path = os.path.join(carpeta, archivo_seleccionado)
    try:
        with open(java_path, 'r', encoding='utf-8') as f:
            codigo_java = f.read()
    except Exception as e:
        codigo_java = f"Error al cargar el archivo Java:\n{e}"
    
    # Cargar el archivo HTML correspondiente
    base = archivo_seleccionado.rsplit('.', 1)[0]
    html_file = base + ".html"
    html_path = os.path.join(carpeta, html_file)
    if os.path.exists(html_path):
        try:
            with open(html_path, 'r', encoding='utf-8') as f:
                codigo_html = f.read()
            # Reemplazar <br> por saltos de línea
            codigo_html = codigo_html.replace("<br>", "\n")
        except Exception as e:
            codigo_html = f"Error al cargar el archivo HTML:\n{e}"
    else:
        codigo_html = "<p>No se encontró el archivo HTML correspondiente.</p>"
    
    try:
        nota = obtener_nota(carpeta, archivo_seleccionado)
    except Exception as e:
        nota = ""
    
    return codigo_java, codigo_html, nota

def refrescar_html(html_code: str, archivo_seleccionado: str, carpeta: str):
    base = archivo_seleccionado.rsplit('.', 1)[0]
    html_file = base + ".html"
    html_path = os.path.join(carpeta, html_file)
    try:
        with open(html_path, "w", encoding="utf-8") as f:
            f.write(html_code)
    except Exception as e:
        print(f"Error al guardar el HTML: {e}")
    # Envolver en un <div> con estilos para ajustar el contenido
    html_render_value = (
        "<div style='white-space: pre-wrap; word-wrap: break-word; overflow-x: hidden;'>"
        f"{html_code}"
        "</div>"
    )
    return html_render_value

def cargar_csv(carpeta: str):
    csv_filename = os.path.join(carpeta, fichero_correciones_csv)
    if not os.path.exists(csv_filename):
         raise ValueError(f"El fichero {fichero_correciones_csv} no existe.")
    df = pd.read_csv(csv_filename, encoding='utf-8-sig')
    if "Nombre completo" not in df.columns or "Calificación" not in df.columns:
         raise ValueError(f"El fichero {fichero_correciones_csv} no tiene las columnas requeridas.")
    df["Calificación"] = df["Calificación"].apply(
        lambda x: format_grade(str(x)) if pd.notnull(x) and str(x).strip() != "" else x
    )
    df = df[["Nombre completo", "Calificación"]].sort_values(by="Nombre completo")
    return df

def guardar_csv_completo(nota: str, archivo_seleccionado: str, html_code: str, carpeta: str):
    csv_filename = os.path.join(carpeta, fichero_correciones_csv)
    if not os.path.exists(csv_filename):
        raise ValueError("El fichero {fichero_correciones_csv} no existe.")
    df_full = pd.read_csv(csv_filename, encoding='utf-8-sig')
    student_name = archivo_seleccionado.split("_")[0].strip()
    norm_student = normalize_string(student_name)
    norm_names_full = df_full["Nombre completo"].apply(lambda x: normalize_string(str(x)))
    if norm_student not in norm_names_full.values:
        raise ValueError(f"El estudiante '{student_name}' no se encontró en el CSV completo.")
    new_cal_formatted = format_grade(str(nota))
    df_full.loc[norm_names_full == norm_student, "Calificación"] = new_cal_formatted
    html_csv = html_code.replace("\n", "<br>")
    df_full.loc[norm_names_full == norm_student, "Comentarios de retroalimentación del profesor"] = html_csv
    df_full.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    # Retornamos el mensaje con el último modificado
    return f"Último modificado: {student_name} ({new_cal_formatted})"

def cargar_solucion(carpeta: str):
    solucion_path = os.path.join(carpeta, f"Solucion{lenguaje_programacion}")
    print(f"Solution path: {solucion_path}")
    if os.path.exists(solucion_path):
        try:
            with open(solucion_path, 'r', encoding='utf-8') as f:
                solucion_code = f.read()
        except Exception as e:
            solucion_code = f"Error al cargar el archivo Solucion{lenguaje_programacion}:\n{e}"
    else:
        solucion_code = f"No se encontró el archivo Solucion{lenguaje_programacion}."
    return solucion_code

### INTERFAZ CON GRADIO (con CSS personalizado) ###

css_personalizado = """
/* Ajuste para los componentes Code y HTML para evitar scroll horizontal */
.gradio-code pre,
.gradio-code textarea,
.gradio-html {
    white-space: pre-wrap !important;
    word-wrap: break-word !important;
    overflow-x: hidden !important;
}
"""

with gr.Blocks(css=css_personalizado, fill_width=True) as demo:
    gr.Markdown("## Coevaluador basado en IA")
    
    # Fila superior: controles de selección
    with gr.Row():
         carpeta_input = gr.Textbox(label="Ruta de la carpeta", placeholder="Ingresa la ruta de la carpeta", value=".", scale=2)
         dropdown_prueba = gr.Dropdown(label=f"Selecciona archivo{lenguaje_programacion}", choices=[], value=None, scale=6)
         nota_estudiante = gr.Textbox(label="Nota del estudiante", interactive=True, scale=2)
         # Se eliminó el botón de guardar CSV
         estado_csv = gr.Textbox(label="Estado CSV", interactive=False, scale=2)
    
    # Fila con dos columnas:
    # Columna izquierda: Examen del estudiante y Solución
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Examen del Estudiante (no editable)")
            java_component = gr.Code(value="", language="cpp", interactive=False, max_lines=maximo_lineas)
            gr.Markdown("### Solución (no editable)")
            solucion_component = gr.Code(value="", language="cpp", interactive=False, max_lines=maximo_lineas)
        # Columna derecha: Feedback en HTML
        with gr.Column(scale=1):
            gr.Markdown("### Feedback al estudiante (HTML)")
            with gr.Tabs() as tabs_html:
                with gr.TabItem("Vista"):
                    html_render = gr.HTML(value="", elem_id="html_render")
                with gr.TabItem("Editor"):
                    html_editor = gr.Code(value="", language="html", interactive=True, max_lines=maximo_lineas)
    
    # Eventos:
    # Al cambiar la carpeta:
    # 1. Se listan los archivos (excluyendo Solucion) y se actualiza la nota.
    # 2. Se carga el fichero Solucion.
    carpeta_input.change(
        fn=listar_archivos_prueba, 
        inputs=carpeta_input, 
        outputs=[dropdown_prueba, nota_estudiante]
    )
    carpeta_input.change(
        fn=cargar_solucion,
        inputs=carpeta_input,
        outputs=solucion_component
    )
    
    # Al cambiar el archivo prueba seleccionado se cargan el examen del estudiante y el HTML correspondiente.
    dropdown_prueba.change(
        fn=cargar_archivos_desde_carpeta, 
        inputs=[carpeta_input, dropdown_prueba],
        outputs=[java_component, html_editor, nota_estudiante]
    ).then(
        fn=lambda html_code, archivo, carpeta: refrescar_html(html_code, archivo, carpeta),
        inputs=[html_editor, dropdown_prueba, carpeta_input],
        outputs=html_render
    )
    
    # Al modificar el HTML en el editor:
    # 1. Se actualiza la vista.
    # 2. Luego se guarda el CSV automáticamente y se actualiza el estado.
    html_editor.change(
        fn=refrescar_html,
        inputs=[html_editor, dropdown_prueba, carpeta_input],
        outputs=html_render
    ).then(
        fn=guardar_csv_completo,
        inputs=[nota_estudiante, dropdown_prueba, html_editor, carpeta_input],
        outputs=estado_csv
    )
    
    # Al modificar la nota del estudiante se guarda el CSV y se actualiza el estado.
    nota_estudiante.change(
        fn=guardar_csv_completo,
        inputs=[nota_estudiante, dropdown_prueba, html_editor, carpeta_input],
        outputs=estado_csv
    )

demo.launch()
